In [ ]:
from google.colab import files
import io
import random
import pandas as pd
import numpy as np

# 강의실 저장할 데이터 프레임 생성
df = pd.DataFrame(columns=['building','room', 'max','plugs','ac_bool','table','chair'])

# 불러와야 할 파일 
# 1. 전체 강의목록 전처리 최종.txt
# 2. 건물-강의실-최대수용인원 최종.txt
class_list = pd.read_csv('/content/전체 강의목록 전처리 최종.txt', delimiter='\t')
classroom_max = pd.read_csv('/content/건물-강의실-최대수용인원 최종.txt', delimiter=',')

# 각 강의실 당 총 이용시간 구하기
class_list.astype({'end': 'int', 'start':'int'})
class_list['using_time'] = class_list['end'] - class_list['start']
class_list.loc[class_list.using_time < 0,'end']= 2300
class_list['using_time'] = class_list['end'] - class_list['start']

sum_time = class_list[['building', 'room', 'using_time']].groupby(by=['building',  'room'], as_index=False).sum()

In [ ]:
# 강의실 별 이용시간, 강의실 크기 정규화 (강의실 수용인원이 20명보다 많은 강의실만 고려)
df_join = pd.merge(sum_time, classroom_max, how='left', on=['building', 'room'])
df_join['max'] = df_join['max'].fillna(-1).astype(int)
idx_delete = df_join[df_join['max'] < 20 ].index
df_join = df_join.drop(idx_delete)
df_join.sort_values(by=['building'], axis=0)

df_join['using_time_norm'] = (df_join['using_time'] - df_join['using_time'].min()) / (df_join['using_time'].max() - df_join['using_time'].min())
df_join['max_norm'] = (df_join['max'] - df_join['max'].min()) / (df_join['max'].max() - df_join['max'].min())
df_join['score'] = df_join['max_norm'] - df_join['using_time_norm']
df_join



,building,room,using_time,max,using_time_norm,max_norm
0,CJ법학관,314,4240,52,0.492857,0.061896
1,CJ법학관,315,3650,62,0.422619,0.081238
2,CJ법학관,316,3380,31,0.390476,0.021277
4,L-P,106,3980,74,0.461905,0.104449
5,L-P,107,3280,75,0.378571,0.106383
...,...,...,...,...,...,...
463,현대자동차경영관,B202,2340,57,0.266667,0.071567
464,현대자동차경영관,B204,3570,88,0.413095,0.131528
465,현대자동차경영관,B205,3500,99,0.404762,0.152805
466,현대자동차경영관,B206,3140,90,0.361905,0.135397


In [ ]:
# 강의실 별 score 구하기
df_join['score'] = df_join['max_norm'] - df_join['using_time_norm']
df_join

,building,room,using_time,max,using_time_norm,max_norm,score
0,CJ법학관,314,4240,52,0.492857,0.061896,-0.430962
1,CJ법학관,315,3650,62,0.422619,0.081238,-0.341381
2,CJ법학관,316,3380,31,0.390476,0.021277,-0.369200
4,L-P,106,3980,74,0.461905,0.104449,-0.357456
5,L-P,107,3280,75,0.378571,0.106383,-0.272188
...,...,...,...,...,...,...,...
463,현대자동차경영관,B202,2340,57,0.266667,0.071567,-0.195100
464,현대자동차경영관,B204,3570,88,0.413095,0.131528,-0.281567
465,현대자동차경영관,B205,3500,99,0.404762,0.152805,-0.251957
466,현대자동차경영관,B206,3140,90,0.361905,0.135397,-0.226508


In [ ]:
# 각 건물별로 점수 높은 강의실 3개씩 선정(건물에 강의실이 3개 미만인 경우 존재하는 개수 만큼 선정)
building_top = df_join.sort_values(by=['building','score'], axis=0, ascending=[True,False]).groupby('building').head(3)
index = np.array(range(len(building_top2))).tolist()
building_top.insert(0,'classroom_index',index)
building_top

,classroom_index,building,room,using_time,max,using_time_norm,max_norm,score
1,0,CJ법학관,315,3650,62,0.422619,0.081238,-0.341381
2,1,CJ법학관,316,3380,31,0.390476,0.021277,-0.369200
0,2,CJ법학관,314,4240,52,0.492857,0.061896,-0.430962
23,3,L-P,436(SUPEX HALL),2170,240,0.246429,0.425532,0.179103
7,4,L-P,205,900,46,0.095238,0.050290,-0.044948
...,...,...,...,...,...,...,...,...
456,77,하나스퀘어,B120,3340,81,0.385714,0.117988,-0.267726
455,78,하나스퀘어,B119,3340,33,0.385714,0.025145,-0.360569
463,79,현대자동차경영관,B202,2340,57,0.266667,0.071567,-0.195100
462,80,현대자동차경영관,B201,2500,59,0.285714,0.075435,-0.210279


In [ ]:
# 추천 대상으로 선정된 강의실에 대한 가상 강의실 데이터 생성

final = building_top[['classroom_index', 'building',	'room', 'max']].copy()
final.reset_index(drop=True, inplace=True)


def generate_plugs(max):
  plugs = []
  for i in range(len(final)):
    plug = random.randrange(0,max+1)
    plugs.append(plug)
  return plugs

def generate_bool(p):
  lounge_desk_height = random.choices([1, 0], [p,1-p], k=len(final))
  return lounge_desk_height

final['plugs'] = generate_plugs(m).copy()
final['ac_bool'] = generate_bool(0.8).copy()
final['table'] = generate_bool(0.5).copy()
final['chair'] = generate_bool(0.7).copy()


final

,classroom_index,building,room,max,plugs,ac_bool,table,chair
0,0,CJ법학관,315,62,19,1,0,1
1,1,CJ법학관,316,31,84,1,1,1
2,2,CJ법학관,314,52,65,1,1,0
3,3,L-P,436(SUPEX HALL),240,5,1,0,1
4,4,L-P,205,46,58,1,1,0
...,...,...,...,...,...,...,...,...
77,77,하나스퀘어,B120,81,61,1,0,1
78,78,하나스퀘어,B119,33,16,1,0,0
79,79,현대자동차경영관,B202,57,55,1,0,0
80,80,현대자동차경영관,B201,59,53,1,0,1


In [ ]:
# 가상 강의실 파일 저장
final.to_csv('강의실 top3 최종.csv', index=False)